# imports

In [40]:
library(tidyverse)
library(dplyr)
library(data.table)
library(caret)
library(Metrics)

In [3]:
cl_data <- read_csv("../datasets/Dataset 3 Beijing/cleaned_beijing.csv")
cl_data <- cl_data[ -c(1) ]
head(cl_data)

Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  .default = col_double(),
  tradeTime = col_date(format = ""),
  buildingType = col_character(),
  renovationCondition = col_character(),
  buildingStructure = col_character(),
  elevator = col_character(),
  fiveYearsProperty = col_character(),
  subway = col_character(),
  district = col_character()
)
See spec(...) for full column specifications.


Lng,Lat,Cid,tradeTime,DOM,followers,totalPrice,price,square,livingRoom,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
116.4755,40.01952,1.111027e+12,2016-08-09,1464,106,415.0,31680,131.00,2,...,Tower,2005,Simplicit,Steel/Concrete,0.217,Has_Elevator,Ownership > 5y,Has_Subway,ChaoYang,56021
116.4539,39.88153,1.111027e+12,2016-07-28,903,126,575.0,43436,132.38,2,...,Tower,2004,Hardcover,Steel/Concrete,0.667,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,71539
116.5620,39.87714,1.111041e+12,2016-12-11,1271,48,1030.0,52021,198.00,3,...,Plate,2005,Simplicit,Steel/Concrete,0.500,Has_Elevator,Ownership > 5y,No_Subway,ChaoYang,48160
116.4380,40.07611,1.111043e+12,2016-09-30,965,138,297.5,22202,134.00,3,...,Tower,2008,Other,Steel/Concrete,0.273,Has_Elevator,Ownership > 5y,No_Subway,ChangPing,51238
116.4284,39.88623,1.111027e+12,2016-08-28,927,286,392.0,48396,81.00,2,...,Plate,1960,Rough,Mixed,0.333,No_elevator,Ownership < 5y,Has_Subway,DongCheng,62588
116.4663,39.99136,1.111027e+12,2016-07-22,861,57,275.6,52000,53.00,1,...,Plate,2005,Simplicit,Steel/Concrete,0.333,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,67738


In [4]:
summary(cl_data)

      Lng             Lat             Cid              tradeTime         
 Min.   :116.1   Min.   :39.63   Min.   :1.111e+12   Min.   :2002-06-01  
 1st Qu.:116.3   1st Qu.:39.89   1st Qu.:1.111e+12   1st Qu.:2013-12-26  
 Median :116.4   Median :39.93   Median :1.111e+12   Median :2015-09-27  
 Mean   :116.4   Mean   :39.95   Mean   :1.126e+12   Mean   :2015-04-30  
 3rd Qu.:116.5   3rd Qu.:40.00   3rd Qu.:1.111e+12   3rd Qu.:2016-08-12  
 Max.   :116.7   Max.   :40.25   Max.   :1.185e+14   Max.   :2018-01-28  
      DOM            followers         totalPrice         price       
 Min.   :   1.00   Min.   :   0.00   Min.   :  13.0   Min.   : 10000  
 1st Qu.:   7.00   1st Qu.:   0.00   1st Qu.: 206.0   1st Qu.: 28362  
 Median :   7.00   Median :   5.00   Median : 295.0   Median : 39133  
 Mean   :  18.54   Mean   :  17.25   Mean   : 350.6   Mean   : 44003  
 3rd Qu.:   8.00   3rd Qu.:  19.00   3rd Qu.: 428.0   3rd Qu.: 54348  
 Max.   :1677.00   Max.   :1143.00   Max.   :4900.0   Ma

In [5]:
unique(cl_data$constructionTime)

[1] 2005 2004 2008 1960 1997 2009 1991 2001 1990 2011 2000 1998 2010 1996 1993
[16] 2006 2002 2012 1989 2003 2007 1994 1984 1992 2014 1985 1999 1979 1981 1976
[31] 1982 1975 1983 1986 1995 1965 2013 1988 1987 2015 1955 1980 1978 1958 1970
[46] 1956 1977 1964 1963 1967 2016 1974 1973 1959 1954 1962 1966 1957 1972 1971
[61] 1953 1968 1961 1950 1952 1969

removing  Cid and total price. cid has too many factors (computer crashes when trained using as factor) 

In [6]:
data = cl_data[-c(3,7)]

# Data After Modifiation

In [7]:
head(data)

Lng,Lat,tradeTime,DOM,followers,price,square,livingRoom,drawingRoom,kitchen,...,buildingType,constructionTime,renovationCondition,buildingStructure,ladderRatio,elevator,fiveYearsProperty,subway,district,communityAverage
116.4755,40.01952,2016-08-09,1464,106,31680,131.00,2,1,1,...,Tower,2005,Simplicit,Steel/Concrete,0.217,Has_Elevator,Ownership > 5y,Has_Subway,ChaoYang,56021
116.4539,39.88153,2016-07-28,903,126,43436,132.38,2,2,1,...,Tower,2004,Hardcover,Steel/Concrete,0.667,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,71539
116.5620,39.87714,2016-12-11,1271,48,52021,198.00,3,2,1,...,Plate,2005,Simplicit,Steel/Concrete,0.500,Has_Elevator,Ownership > 5y,No_Subway,ChaoYang,48160
116.4380,40.07611,2016-09-30,965,138,22202,134.00,3,1,1,...,Tower,2008,Other,Steel/Concrete,0.273,Has_Elevator,Ownership > 5y,No_Subway,ChangPing,51238
116.4284,39.88623,2016-08-28,927,286,48396,81.00,2,1,1,...,Plate,1960,Rough,Mixed,0.333,No_elevator,Ownership < 5y,Has_Subway,DongCheng,62588
116.4663,39.99136,2016-07-22,861,57,52000,53.00,1,0,1,...,Plate,2005,Simplicit,Steel/Concrete,0.333,Has_Elevator,Ownership < 5y,No_Subway,ChaoYang,67738


In [8]:
str(data)

Classes 'tbl_df', 'tbl' and 'data.frame':	296350 obs. of  22 variables:
 $ Lng                : num  116 116 117 116 116 ...
 $ Lat                : num  40 39.9 39.9 40.1 39.9 ...
 $ tradeTime          : Date, format: "2016-08-09" "2016-07-28" ...
 $ DOM                : num  1464 903 1271 965 927 ...
 $ followers          : num  106 126 48 138 286 57 167 138 218 134 ...
 $ price              : num  31680 43436 52021 22202 48396 ...
 $ square             : num  131 132 198 134 81 ...
 $ livingRoom         : num  2 2 3 3 2 1 2 3 1 1 ...
 $ drawingRoom        : num  1 2 2 1 1 0 1 2 0 0 ...
 $ kitchen            : num  1 1 1 1 1 1 1 1 1 0 ...
 $ bathRoom           : num  1 2 3 1 1 1 1 2 1 0 ...
 $ floor              : num  26 22 4 21 6 8 6 22 10 23 ...
 $ buildingType       : chr  "Tower" "Tower" "Plate" "Tower" ...
 $ constructionTime   : num  2005 2004 2005 2008 1960 ...
 $ renovationCondition: chr  "Simplicit" "Hardcover" "Simplicit" "Other" ...
 $ buildingStructure  : chr  "Steel/Con

# Runing an analysis of variabce to see the significat factors

In [9]:
anova <- aov( price ~ . , data = data )

In [10]:
summary(anova)

                        Df    Sum Sq   Mean Sq  F value Pr(>F)    
Lng                      1 3.362e+12 3.362e+12  35191.6 <2e-16 ***
Lat                      1 2.696e+11 2.696e+11   2822.3 <2e-16 ***
tradeTime                1 3.956e+13 3.956e+13 414056.7 <2e-16 ***
DOM                      1 9.808e+11 9.808e+11  10266.2 <2e-16 ***
followers                1 1.049e+10 1.049e+10    109.8 <2e-16 ***
square                   1 3.485e+12 3.485e+12  36475.4 <2e-16 ***
livingRoom               1 3.904e+11 3.904e+11   4086.1 <2e-16 ***
drawingRoom              1 5.909e+10 5.909e+10    618.4 <2e-16 ***
kitchen                  1 1.609e+11 1.609e+11   1683.8 <2e-16 ***
bathRoom                 1 4.947e+11 4.947e+11   5178.2 <2e-16 ***
floor                    1 5.533e+11 5.533e+11   5791.1 <2e-16 ***
buildingType             3 5.894e+11 1.965e+11   2056.5 <2e-16 ***
constructionTime         1 5.432e+12 5.432e+12  56851.1 <2e-16 ***
renovationCondition      3 1.355e+12 4.517e+11   4727.3 <2e-16

# Splitting Data

In [41]:
inTrain = createDataPartition(data$price, p = 0.7, list = F)
train = data[inTrain,]
test = data[-inTrain,]

# Model creation
## model with liner combination of all facotors

In [42]:
model <- train(price ~ . , method = "glm", data = train )

In [43]:
summary(model)


Call:
NULL

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-90650   -6067    -602    5453  132949  

Coefficients:
                                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)                        2.495e+05  6.211e+04   4.017 5.90e-05 ***
Lng                               -3.155e+03  4.658e+02  -6.774 1.26e-11 ***
Lat                               -3.427e+03  5.341e+02  -6.416 1.40e-10 ***
tradeTime                          2.264e+01  5.500e-02 411.613  < 2e-16 ***
DOM                                3.631e+01  6.761e-01  53.697  < 2e-16 ***
followers                          8.383e+00  7.480e-01  11.208  < 2e-16 ***
square                            -6.292e+01  1.324e+00 -47.515  < 2e-16 ***
livingRoom                         1.157e+02  4.562e+01   2.535 0.011244 *  
drawingRoom                        1.239e+03  5.481e+01  22.615  < 2e-16 ***
kitchen                           -9.388e+02  2.277e+02  -4.122 3.75e-05 ***
bathRoom              

In [44]:
p = predict(model, test)
error = sqrt(sum((p-test$price)^2)/length(p))
error

[1] 9800.42

## model with limited factors for initial testing

In [46]:
model_limited <- train(price ~ square + constructionTime + subway + district , method = "glm", data = train )

In [47]:
summary(model_limited)


Call:
NULL

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-60762  -11264   -3488    9175  114259  

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)         46000.893  10025.804   4.588 4.47e-06 ***
square                -33.539      1.186 -28.287  < 2e-16 ***
constructionTime       -5.252      5.025  -1.045 0.295898    
subwayNo_Subway     -3719.420     84.313 -44.114  < 2e-16 ***
districtChaoYang    12389.414    133.839  92.569  < 2e-16 ***
districtDaXing       1258.739    467.794   2.691 0.007129 ** 
districtDongCheng   29152.687    207.837 140.267  < 2e-16 ***
districtFangShang   -1536.650    428.800  -3.584 0.000339 ***
districtFaXing       -172.629    208.928  -0.826 0.408656    
districtFengTai      7116.206    171.055  41.602  < 2e-16 ***
districtHaiDian     24047.719    162.253 148.211  < 2e-16 ***
districtMenTouGou   -5277.102    251.883 -20.951  < 2e-16 ***
districtShiJingShan  5076.134    238.943  21.244  < 2e-16 ***
d

In [48]:
p = predict(model_limited, test)
error = sqrt(sum((p-test$price)^2)/length(p))
error

[1] 17665.7

In [65]:
final_model = model_limited$finalModel

In [66]:
class(final_model)

[1] "glm" "lm"

## Cleaning the model

In [67]:
object.size(final_model)

212518816 bytes

In [68]:
strip_glm <- function(cm) {
  cm$y = c()
  cm$model = c()
  
  cm$residuals = c()
  cm$fitted.values = c()
  cm$effects = c()
  cm$qr$qr = c()  
  cm$linear.predictors = c()
  cm$weights = c()
  cm$prior.weights = c()
  cm$data = c()

  
  cm$family$variance = c()
  cm$family$dev.resids = c()
  cm$family$aic = c()
  cm$family$validmu = c()
  cm$family$simulate = c()
  attr(cm$terms,".Environment") = c()
  attr(cm$formula,".Environment") = c()
  
  cm
}
final_model_cleaned = strip_glm(final_model)

In [69]:
object.size(final_model_cleaned)

40536 bytes

## Saving the model to disk


In [70]:
saveRDS(final_model_cleaned, "./models/final_model.rds")
saveRDS(final_model_cleaned, "../house-price-prediction-app/models/final_model.rds")

verifying saved model

In [73]:
mdl = readRDS("./models/final_model.rds")
class(mdl)

[1] "glm" "lm"